In [4]:
import networkx as nx
import ipywidgets as widgets
from ipywidgets import interact
from pyvis.network import Network
from IPython.display import display, HTML
import os
import pickle 


In [5]:

%matplotlib inline


In [6]:
# Define the ordered list of scale degree labels.
scale_degrees = [
    'A2', 'm2', 'P8', 'A6',
    'm7', 'M2', 'm6', 'M7',
    'm3', 'M3', 'P5', 'd7',
    'P4', 'M6', 'A4', 'd5'
]
n_scale = len(scale_degrees)

INTERVAL_TO_SEMITONES = {
    'P8': 0,   # tonic (0 semitones)
    'm2': 1,   # 1 semitone
    'M2': 2,   # 2 semitones
    'm3': 3,   # 3 semitones
    'A2': 3,   # augmented 2nd (enharmonically same as m3)
    'M3': 4,   # 4 semitones
    'P4': 5,   # 5 semitones
    'A4': 6,   # augmented fourth (6 semitones)
    'd5': 6,   # diminished fifth (6 semitones)
    'P5': 7,   # 7 semitones
    'm6': 8,   # 8 semitones
    'M6': 9,   # 9 semitones
    'd7': 9,   # diminished seventh (9 semitones)
    'm7': 10,  # 10 semitones
    'A6': 10,  # augmented sixth (10 semitones)
    'M7': 11   # 11 semitones
}
rest_val = 16

In [4]:
with open('scale_degree_map.pkl', 'rb') as f:
    scale_map = pickle.load(f)    
print(scale_map)

{'A2': 0, 'm2': 1, 'P8': 2, 'A6': 3, 'm7': 4, 'M2': 5, 'm6': 6, 'M7': 7, 'm3': 8, 'M3': 9, 'P5': 10, 'd7': 11, 'P4': 12, 'M6': 13, 'A4': 14, 'd5': 15}


In [7]:

def parse_graph_from_lines(lines):
    """
    Parse a single graph from a list of nonempty lines.
    
    Expected format:
       N=50
       X:
       <node values spanning one or more lines>
       E:
       <N*N edge values spanning one or more lines>
    """
    N = int(lines[0].split("=")[1])
    try:
        x_index = lines.index("X:") + 1
    except ValueError:
        raise ValueError("Could not find 'X:' section in the graph chunk.")
    try:
        e_index = lines.index("E:")
    except ValueError:
        raise ValueError("Could not find 'E:' section in the graph chunk.")
    
    x_values = []
    for line in lines[x_index:e_index]:
        x_values.extend(line.split())
    if len(x_values) != N:
        raise ValueError(f"Expected {N} node values but found {len(x_values)}")
    nodes = [int(val) for val in x_values]
    
    edge_numbers = []
    for line in lines[e_index+1:]:
        edge_numbers.extend(line.split())
    if len(edge_numbers) != N * N:
        raise ValueError(f"Expected {N*N} edge values but found {len(edge_numbers)}")
    
    matrix = []
    for i in range(N):
        row = [int(edge_numbers[i * N + j]) for j in range(N)]
        matrix.append(row)

    upper_triangular = [
        [matrix[i][j] if i <= j else 0 for j in range(len(matrix[0]))]
        for i in range(len(matrix))
    ]
    
    return N, nodes, upper_triangular

def parse_graphs_file(filename):
    """
    Parse a file that contains one or more graphs.
    Each graph is assumed to start with a line beginning with "N=".
    Returns a list of (N, nodes, matrix) tuples.
    """
    with open(filename, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
    
    graphs_data = []
    i = 0
    while i < len(lines):
        if lines[i].startswith("N="):
            current_chunk = []
            # Collect lines until we hit the next "N=" (except for the first line)
            while i < len(lines) and (not (lines[i].startswith("N=") and current_chunk)):
                current_chunk.append(lines[i])
                i += 1
            graphs_data.append(parse_graph_from_lines(current_chunk))
        else:
            i += 1
    return graphs_data

def build_graph(N, node_vals, matrix):
    """
    Build a NetworkX directed graph from node values and the edge matrix.
    Node values of 51 are treated as a rest.
    """
    G = nx.DiGraph()
    for i, val in enumerate(node_vals):
        label = "Rest" if val == rest_val else scale_degrees[val % n_scale]
        G.add_node(i, label=label)
    for i in range(N):
        for j in range(N):
            w = matrix[i][j]
            if w != 0:
                G.add_edge(i, j, weight=w)
    return G


In [9]:

# Parse all graphs from the file.
filename = "generated_samples1.txt"
graphs_data = parse_graphs_file(filename)

# Create widgets for interactive selection.
graph_dropdown = widgets.Dropdown(
    options=[(f"Graph {i}", i) for i in range(len(graphs_data))],
    description="Graph:"
)
threshold_slider = widgets.IntSlider(
    value=0, min=0, max=30, step=1, description="Min Edge Weight:"
)

def update_pyvis(graph_index, threshold):
    """
    Update the interactive PyVis visualization.
    Filters out edges with weight below the given threshold.
    """
    N, node_vals, matrix = graphs_data[graph_index]
    G = build_graph(N, node_vals, matrix)
    
    # Create a filtered copy of the graph.
    H = nx.DiGraph()
    for node, data in G.nodes(data=True):
        H.add_node(node, **data)
    for u, v, data in G.edges(data=True):
        if data['weight'] >= threshold:
            H.add_edge(u, v, **data)
    
    # Create a PyVis network.
    net = Network(height="600px", width="100%", directed=True, notebook=True, cdn_resources='in_line')

    net.from_nx(H)
    
    # Save to a temporary HTML file and display inline.
    temp_html = "temp_graph.html"
    net.show(temp_html)
    with open(temp_html, "r", encoding="utf-8") as f:
        html_content = f.read()
    # display(HTML(html_content))
 




In [11]:
def get_note_sequence(G):
    """
    Traverse G as a chain: starting from node 0, follow the unique outgoing edge
    (choose the smallest index successor) from each node.
    Returns a list of node indices representing the note sequence.
    """
    sequence = []
    current = 0
    visited = set()
    while True:
        if current in visited:
            break
        visited.add(current)
        sequence.append(current)
        successors = [j for j in G.successors(current) if j > current]
        if not successors:
            break
        next_node = min(successors)
        current = next_node
    return sequence

In [10]:
# Create interactive controls.
interact(update_pyvis, graph_index=graph_dropdown, threshold=threshold_slider)

interactive(children=(Dropdown(description='Graph:', options=(('Graph 0', 0), ('Graph 1', 1), ('Graph 2', 2)),…

<function __main__.update_pyvis(graph_index, threshold)>

In [ ]:
import panel as pn
import param

pn.extension()
def update_network_html(graph_index, threshold, graphs_data):
    """
    Given a graph index and edge weight threshold, build the corresponding graph,
    filter out edges with weight < threshold, and generate a PyVis network.
    Returns the HTML content as a string.
    """
    N, node_values, matrix = graphs_data[graph_index]
    G = build_graph(N, node_values, matrix)
    
    # Filter graph: create a new graph with only edges meeting the threshold.
    H = nx.DiGraph()
    for node, data in G.nodes(data=True):
        H.add_node(node, **data)
    for u, v, data in G.edges(data=True):
        if data['weight'] >= threshold:
            H.add_edge(u, v, **data)
    
    net = Network(height="600px", width="100%", directed=True, cdn_resources='in_line')
    net.from_nx(H)
    # Save to a temporary HTML file.
    temp_html = "temp_network.html"
    net.show(temp_html)
    with open(temp_html, "r", encoding="utf-8") as f:
        html_content = f.read()
    os.remove(temp_html)
    return html_content

# --- Panel Dashboard Definition ---

class Dashboard(param.Parameterized):
    graph_index = param.Integer(default=0, bounds=(0, 0))
    threshold = param.Integer(default=0, bounds=(0, 30))
    
    def view_network(self):
        return pn.pane.HTML(update_network_html(self.graph_index, self.threshold, graphs_data),
                              sizing_mode="stretch_both")

In [21]:
filename = "generated_samples1.txt"
graphs_data = parse_graphs_file(filename)
num_graphs = len(graphs_data)
if num_graphs == 0:
    print("No graphs found in the file.")


# Create the dashboard and update parameter bounds.
dashboard = Dashboard()
dashboard.param.graph_index.bounds = (0, num_graphs - 1)

# Create a Panel layout with parameter widgets and the network view.
dashboard_panel = pn.Row(
    pn.Param(dashboard.param, widgets={
        'graph_index': pn.widgets.IntSlider, 
        'threshold': pn.widgets.IntSlider
    }),
    dashboard.view_network
)
# Save the dashboard as a standalone HTML file.
output_html = "dashboard.html"
dashboard_panel.save(output_html, title="Graph Dashboard", embed=True)
print("Dashboard saved as", output_html)

temp_network.html


AttributeError: 'NoneType' object has no attribute 'render'